<a href="https://colab.research.google.com/github/njokinjuguna/Machine-learning-Models/blob/main/CancerPredictiveAnalysisInKenya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Path to the KNH workbook
knh_workbook_path = 'KNH_Data.xlsx'  # Replace with the actual path

# Load the workbook
knh_data = pd.ExcelFile(knh_workbook_path)

# Loop through each sheet in the workbook and display its structure and content
for sheet_name in knh_data.sheet_names:
    print(f"\nSheet name: {sheet_name}")

    # Load the sheet into a DataFrame
    sheet_data = knh_data.parse(sheet_name)

    # Display basic information about the sheet
    print("Structure:")
    print(sheet_data.info())

    # Display the first few rows of the sheet
    print("First few rows:")
    print(sheet_data.head())

# Optional: Close the workbook if necessary
knh_data.close()



Sheet name: Original Database 
Structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 56 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Questionnaire No                                             500 non-null    object 
 1   Site                                                         500 non-null    object 
 2   Age                                                          500 non-null    int64  
 3   Sex                                                          500 non-null    object 
 4   Residence                                                    500 non-null    object 
 5   Referral                                                     500 non-null    object 
 6   Referred From                                                331 non-null    object 
 7   Gastric cancer                       

In [ ]:
import pandas as pd

# Path to the MTRH workbook
mtrh_workbook_path = 'MTRH_Data.xlsx'  # Replace with the actual path

# Load the workbook
mtrh_data = pd.ExcelFile(mtrh_workbook_path)

# Loop through each sheet in the workbook and display its structure and content
for sheet_name in mtrh_data.sheet_names:
    print(f"\nSheet name: {sheet_name}")

    # Load the sheet into a DataFrame
    sheet_data = mtrh_data.parse(sheet_name)

    # Display basic information about the sheet
    print("Structure:")
    print(sheet_data.info())

    # Display the first few rows of the sheet
    print("First few rows:")
    print(sheet_data.head())

# Optional: Close the workbook if necessary
knh_data.close()


Sheet name: Original Database
Structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 57 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Questionnaire No                                             500 non-null    object 
 1   Site                                                         500 non-null    object 
 2   Age                                                          500 non-null    int64  
 3   Sex                                                          500 non-null    object 
 4   Residence                                                    500 non-null    object 
 5   Referral                                                     496 non-null    object 
 6   Referred From                                                207 non-null    object 
 7   Gastric cancer                        

In [7]:
import pandas as pd

# Define paths to the KNH and MTRH workbooks
knh_workbook_path = 'KNH_Data.xlsx'  # Replace with the actual KNH file path
mtrh_workbook_path = 'MTRH_Data.xlsx'  # Replace with the actual MTRH file path

# Initialize an empty list to store Cleaned DB data from both workbooks
cleaned_db_data = []

# Function to extract the province-to-region mapping from the Coding sheet
def get_province_mapping(coding_sheet):
    # Drop unnecessary rows to get only the mapping part
    mapping_rows = coding_sheet.dropna(subset=['Unnamed: 13', 'Unnamed: 14']).tail(8)  # Tail for last 8 rows
    province_mapping = dict(zip(mapping_rows['Unnamed: 13'], mapping_rows['Unnamed: 14']))
    return province_mapping

# Loop to process both KNH and MTRH workbooks
for workbook_path, hospital_name in [(knh_workbook_path, 'KNH'), (mtrh_workbook_path, 'MTRH')]:
    # Load the workbook
    workbook = pd.ExcelFile(workbook_path)

    # Load the Coding sheet to map provinces to region names
    if 'Coding' in workbook.sheet_names:
        coding_sheet = workbook.parse('Coding')

        # Get province-to-region mapping
        province_mapping = get_province_mapping(coding_sheet)

    # Loop through each sheet to find "Cleaned DB"
    for sheet_name in workbook.sheet_names:
        if sheet_name == 'Cleaned DB':
            # Load the "Cleaned DB" sheet
            cleaned_db = workbook.parse(sheet_name)
            # Add a 'Hospital' column to distinguish the data source
            cleaned_db['Hospital'] = hospital_name
            # Rename columns
            cleaned_db = cleaned_db.rename(columns={
                'Questionnaire No': 'Record ID',
                'Unnamed: 3': 'Gender',
                'Residence': 'Residence',
                'Referral': 'Referral',
                'Referred From': 'Referred From',
                'Province': 'Province'
            })
            # Map Province codes to region names using the Coding sheet
            cleaned_db['Province'] = cleaned_db['Province'].map(province_mapping)
            # Append the data to our list
            cleaned_db_data.append(cleaned_db)
            print(f"Loaded Cleaned DB from {hospital_name} workbook.")

    # Optional: Close the workbook if needed (Pandas automatically handles it usually)
    workbook.close()

# Concatenate all Cleaned DB data into a single DataFrame
combined_cleaned_db = pd.concat(cleaned_db_data, ignore_index=True)

# Display structure and a preview of the combined dataset
print("Combined Cleaned DB Structure:")
print(combined_cleaned_db.info())
print("First few rows of the Combined Dataset:")
print(combined_cleaned_db.head())

# Save the combined dataset if needed
output_file_path = 'Combined_Cleaned_DB.xlsx'  # Specify the desired output path
combined_cleaned_db.to_excel(output_file_path, index=False)

print(f"Combined dataset saved to {output_file_path}")


Loaded Cleaned DB from KNH workbook.
Loaded Cleaned DB from MTRH workbook.
Combined Cleaned DB Structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 65 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Record ID                                                    1000 non-null   object 
 1   Site                                                         1000 non-null   object 
 2   Age                                                          1000 non-null   int64  
 3   Gender                                                       500 non-null    object 
 4   Residence                                                    1000 non-null   object 
 5   Province                                                     1000 non-null   object 
 6   Referral                                                     99

Combined dataset saved to Combined_Cleaned_DB.xlsx
